# Deploy Signature: Signature Model Package from AWS Marketplace

This sample notebook shows you how to deploy Signature Model using Amazon SageMaker.

    Note: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:

1. Note: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that IAM role used has AmazonSageMakerFullAccess
3. To deploy this ML model successfully, ensure that:
   - Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used:
        - aws-marketplace:ViewSubscriptions
        - aws-marketplace:Unsubscribe
        - aws-marketplace:Subscribe
   - or your AWS account has a subscription. If so, skip step: Subscribe to the model package


## Contents:
1. Subscribe to the model package
2. Create an endpoint and perform real-time inference
    - Create an endpoint
    - Create input payload
    - Perform real-time inference
    - Visualize output
    - Delete the endpoint
3. Perform batch inference
4. Clean-up
    - Delete the model
    - Unsubscribe to the listing (optional))

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package
To subscribe to the model package:

1. Open the model package listing page
2. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms.
4. Once you click on **Continue to configuration** button and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.


In [1]:
model_package_arn = model_package_arn #provided after purchase

In [ ]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import numpy as np

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

In [4]:
model_name = "signature"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.m4.xlarge"
)
batch_transform_inference_instance_type = (
    "ml.m4.xlarge"
)

### A. Create an endpoint

In [8]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----!

### B Create input payload

In [9]:
import os
import json
import base64

input_img_pth1 = "/data/input/real-time/real1.jpeg" # Path to image 
input_img_pth2 = "/data/input/real-time/real2.jpg"

# Read the image file and encode it to base64
with open(input_img_pth1, "rb") as img:
    image = base64.b64encode(img.read())
    img1_base64 = image.decode()
    
with open(input_img_pth2, "rb") as img:
    image = base64.b64encode(img.read())
    img2_base64 = image.decode()

# Prepare the data in the expected format
data = {"image1": img1_base64,
       "image2" : img2_base64}

# Write the data to a JSON file
file_name = "input.json"
with open(file_name, "w") as json_file:
    json.dump(data, json_file)

# Set the necessary parameters for invoking the SageMaker endpoint
# model_name = "model"
output_file_name = "data/output/signature_output.txt"
content_type = "application/json"
region = "us-east-1"  # Replace with your actual AWS region, e.g., "us-east-1"


### C perform real time inference

In [10]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://{file_name} --content-type $content_type --region $sagemaker_session.boto_region_name $output_file_name

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


### D Visualise the output

In [11]:
output_file = "data/output/signature_output.txt"  # Path to the output file

# Read the contents of the output file
with open(output_file, "r") as file:
    output_data = file.read()

# Print the output
print(output_data)

{"result":"Similar Signature","similarity_score":0.7038995623588562,"statusCode":200}



### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

### 3. Perform batch inference
In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:

[How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html)

[How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html)

In [ ]:
# upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch/signature"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

In [ ]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

### 4. Clean-up
A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable](https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/models) model created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model.

**Steps to unsubscribe to product from AWS Marketplace:**

- Navigate to Machine Learning tab on Your [Software subscriptions page](https://us-east-1.console.aws.amazon.com/marketplace/home#/landing)

- Locate the listing that you want to cancel the subscription for, and then choose **Cancel Subscription** to cancel the subscription.